# Check ice dates

In [14]:
import xarray as xr
import glob
import cftime
import cosima_cookbook as cc
import pandas as pd
import numpy as np

In [2]:
basedir = '/g/data3/hh5/tmp/cosima/access-om2-01/01deg_jra55v13_iaf/output001/ice-test/OUTPUT/'
basedir = '/g/data3/hh5/tmp/cosima/access-om2-01/01deg_jra55v13_iaf'
dsi = xr.open_dataset(basedir+'/output001/ice/OUTPUT/iceh.1985-01-daily.nc')

# dsi = xr.open_dataset(basedir+'/output001/ice/OUTPUT/iceh.1985-01.nc')

# dso = xr.open_dataset(basedir+'/output001/ocean/ocean.nc')

In [23]:
vicen_m = cc.get_nc_variable('01deg_jra55v13_iaf', 'iceh.????-??.nc', 'vicen_m',n=1)

Using database sqlite:////g/data3/hh5/tmp/cosima/cosima-cookbook/cosima-cookbook.db


/g/data1a/v45/aek156/notebooks/github/OceansAus/cosima-cookbook/cosima_cookbook/netcdf_index.py:446: FutureWarning: The autoclose argument is no longer used by xarray.open_dataset() and is now ignored; it will be removed in xarray v0.12. If necessary, you can control the maximum number of simultaneous open files with xarray.set_options(file_cache_maxsize=...).
  autoclose=True)[variables]


In [24]:
vicen_m.groupby('time.month').mean('time')

<xarray.DataArray 'vicen_m' (month: 1, nc: 5, nj: 2700, ni: 3600)>
dask.array<shape=(1, 5, 2700, 3600), dtype=float32, chunksize=(1, 5, 2700, 3600)>
Coordinates:
  * month    (month) int64 2
Dimensions without coordinates: nc, nj, ni

In [25]:
vicen_m['time'] = vicen_m.time - np.timedelta64(12, 'h')

In [26]:
vicen_m.groupby('time.month').mean('time')

<xarray.DataArray 'vicen_m' (month: 1, nc: 5, nj: 2700, ni: 3600)>
dask.array<shape=(1, 5, 2700, 3600), dtype=float32, chunksize=(1, 5, 2700, 3600)>
Coordinates:
  * month    (month) int64 1
Dimensions without coordinates: nc, nj, ni

In [16]:
vicen_m.time - np.timedelta64(12, 'h') # pd.DateOffset(hours=-12)

<xarray.DataArray 'time' (time: 1)>
array(['1900-01-31T12:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 1900-02-01

In [86]:
type(dsi.time.values[0])

cftime._cftime.DatetimeGregorian

In [67]:
newtime = dsi.time_bounds.astype('int64').mean(axis=1).astype('datetime64[ns]')

In [87]:
newtime = dsi.time_bounds.astype('int64').mean(axis=1).astype(type(dsi.time.values[0]))

In [88]:
newtime

<xarray.DataArray 'time_bounds' (time: 31)>
array([4.734288e+17, 4.735152e+17, 4.736016e+17, 4.73688e+17, 4.737744e+17,
       4.738608e+17, 4.739472e+17, 4.740336e+17, 4.7412e+17, 4.742064e+17,
       4.742928e+17, 4.743792e+17, 4.744656e+17, 4.74552e+17, 4.746384e+17,
       4.747248e+17, 4.748112e+17, 4.748976e+17, 4.74984e+17, 4.750704e+17,
       4.751568e+17, 4.752432e+17, 4.753296e+17, 4.75416e+17, 4.755024e+17,
       4.755888e+17, 4.756752e+17, 4.757616e+17, 4.75848e+17, 4.759344e+17,
       4.760208e+17], dtype=object)
Coordinates:
  * time     (time) object 1985-01-02 00:00:00 ... 1985-02-01 00:00:00

In [82]:
dsi['time'] = newtime

In [83]:
dsi['time']

<xarray.DataArray 'time' (time: 31)>
array([4.734288e+17, 4.735152e+17, 4.736016e+17, 4.73688e+17, 4.737744e+17,
       4.738608e+17, 4.739472e+17, 4.740336e+17, 4.7412e+17, 4.742064e+17,
       4.742928e+17, 4.743792e+17, 4.744656e+17, 4.74552e+17, 4.746384e+17,
       4.747248e+17, 4.748112e+17, 4.748976e+17, 4.74984e+17, 4.750704e+17,
       4.751568e+17, 4.752432e+17, 4.753296e+17, 4.75416e+17, 4.755024e+17,
       4.755888e+17, 4.756752e+17, 4.757616e+17, 4.75848e+17, 4.759344e+17,
       4.760208e+17], dtype=object)
Coordinates:
  * time     (time) object 4.734e+17 4.735e+17 4.736e+17 ... 4.759e+17 4.76e+17

In [10]:
dso.time_bounds

OverflowError: Python int too large to convert to C long

In [3]:
for m in range(3):
    ds_concat = xr.open_dataset(basedir+'iceh.1985-'+str(m+1).zfill(2)+'-daily.nc')
    ndailies = len(glob.glob(basedir+'iceh.1985-'+str(m+1).zfill(2)+'-??.nc'))
    for d in range(ndailies):
        print(m+1, d+1)
        ds_day = xr.open_dataset(basedir+'iceh.1985-'+str(m+1).zfill(2)+'-'+str(d+1).zfill(2)+'.nc')
        # don't check the following
        ds_day.attrs.update({'comment2': ds_concat.comment2})
        ds_day.attrs.update({'history': ds_concat.history})
        ds_day.attrs.update({'NCO': ds_concat.NCO})
        ds_day.attrs.update({'nco_openmp_thread_number': ds_concat.nco_openmp_thread_number})

    #     print(ds_day.isel(time=0).equals(ds_concat.isel(time=d)))
        print(ds_day.isel(time=0).identical(ds_concat.isel(time=d)))

1 1
True
1 2
True
1 3
True
1 4
True
1 5
True
1 6
True
1 7
True
1 8
True
1 9
True
1 10
True
1 11
True
1 12
True
1 13
True
1 14
True
1 15
True
1 16
True
1 17
True
1 18
True
1 19
True
1 20
True
1 21
True
1 22
True
1 23
True
1 24
True
1 25
True
1 26
True
1 27
True
1 28
True
1 29
True
1 30
True
1 31
True
2 1
True
2 2
True
2 3
True
2 4
True
2 5
True
2 6
True
2 7
True
2 8
True
2 9
True
2 10
True
2 11
True
2 12
True
2 13
True
2 14
True
2 15
True
2 16
True
2 17
True
2 18
True
2 19
True
2 20
True
2 21
True
2 22
True
2 23
True
2 24
True
2 25
True
2 26
True
2 27
True
2 28
True
3 1
True
3 2
True
3 3
True
3 4
True
3 5
True
3 6
True
3 7
True
3 8
True
3 9
True
3 10
True
3 11
True
3 12
True
3 13
True
3 14
True
3 15
True
3 16
True
3 17
True
3 18
True
3 19
True
3 20
True
3 21
True
3 22
True
3 23
True
3 24
True
3 25
True
3 26
True
3 27
True
3 28
True
3 29
True
3 30
True
3 31
True
